In [1]:
import numpy as np
from scipy import signal
from scipy.signal import chirp, spectrogram
from scipy import constants
import pint
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button, RadioButtons
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
# Registro de unidades
ureg = pint.UnitRegistry()
pint._DEFAULT_REGISTRY = ureg

def modulo(numero):
    "Funcion auxiliar que calcula el modulo de un numero"
    "recibe una variable de tipo pint.quantity"
    "retorna su módulo"
    if numero.magnitude < 0:
        return -1 * numero
    else:
        return numero

def calculo_parametros(bitrate, lambda_o , dispersion_cromatica, chirp_o, longitud):
    #chirp_o, longitud
    # Constantes de interés
    c_luz = constants.c * (ureg.meter) / (1 * ureg.second)
    pi = constants.pi
    #######################################
    ### Calculo ensanchamiento temporal ###
    # requiere bitrate, retorna T_o #
    bps = bitrate.to_base_units() #bps
    tiempo_de_bit = 1 / (bps/ureg.bit) #s
    T_o = tiempo_de_bit / 2 #s
    #######################################
    ### Calculo beta_2 ###
    #"$\beta_2 = -\frac{\lambda^2}{2\pi c}D$"#
    numerador = -((lambda_o.to_base_units()**2) * dispersion_cromatica.to_base_units())
    denominador = (2*pi*c_luz)
    beta_2 = numerador / denominador
    #######################################
    ### Calculo chirp ###
    #"$k_z = k_o + \frac{(1+k_o^2)\beta_2z}{T_0^2}$"#
    #A = (beta_2.to_base_units() * longitud.to_base_units()) / (T_o**2).to_base_units()
    #chirp_z = chirp_o * (1 + chirp_o**2) + A
    if longitud == (0 * ureg('kilometer')) :
        chirp_z = chirp_o
    else: 
        chirp_z = (beta_2.to_base_units() * longitud.to_base_units()) / (T_o**2).to_base_units() + chirp_o * (1 + chirp_o**2)
    #######################################
    ### Calculo ensanchamiento temporal ###
    #"$\frac{T_z}{T_o} = \sqrt{(1 + \frac{\beta_2zk_o}{To^2})^2+(\frac{\beta_2z}{T_o^2})^2}$"#
    A = (beta_2.to_base_units() * longitud.to_base_units() * chirp_o) / (T_o.to_base_units()**2)
    B = (beta_2.to_base_units() * longitud.to_base_units()) / (T_o.to_base_units()**2)
    T_z = ( ( ((1 + A)**2) + B**2 ) ** (1/2) ) * T_o
    Tz_To = T_z / T_o
    #######################################
    ### Calculo longitud de dispersion ###
    #"$L_D = \frac{2T_o}{\beta_2}$"#
    if beta_2.magnitude == 0 :
        L_Dispersion = float('inf') * ureg('kilometer')
    else:
        mod_beta2 = (modulo(beta_2))
        L_Dispersion = ( (T_o**2) / (mod_beta2) ).to(ureg('kilometer'))
    return T_o, T_z, Tz_To, beta_2, chirp_z, L_Dispersion

def calculo_frecuencias_ploteo(chirp):
    chirp = int(chirp)
    if chirp == 0 :
        f0_plot = 1
        f1_plot = f0_plot
    elif chirp > 0:
        f0_plot = 1 
        f1_plot = chirp
    else:
        f0_plot = chirp 
        f1_plot = 1 
    return f0_plot, f1_plot
   
###########################################
####### Ploteos de prueba, comentar #######
## Prueba plot chirp ###
def plot_chirp(T_o, T_z, chirp_z):
    To_plot = int(T_o.to(ureg('picosecond')).magnitude)
    Tz_plot = int(T_z.to(ureg('picosecond')).magnitude)
    t_plot_chirp = np.linspace(0, Tz_plot/5, To_plot*500)
    f0_plot, f1_plot = calculo_frecuencias_ploteo(chirp_z)
    cos_chirp = chirp(t_plot_chirp, f0=f0_plot, f1=f1_plot, t1=Tz_plot/5)
    return cos_chirp


## Prueba plot envolvente gaussiana ###
def plot_e_gauss(T_o, T_z):
    To_plot = int(T_o.to(ureg('picosecond')).magnitude)
    Tz_plot = int(T_z.to(ureg('picosecond')).magnitude)
    t_plot_gauss = np.linspace(-3*Tz_plot, 3*Tz_plot, To_plot*500)
    A = round(T_o/T_z, 4)
    i, q, env = A * signal.gausspulse(t_plot_gauss, fc=1/Tz_plot, retquad=True, retenv=True)
    return t_plot_gauss, env

#plot_e_gauss(T_o, T_z)
###########################################


### ¡¡FINALMENTE!! ###
def plot_pulso_gaussiano(T_o, T_z, chirp_o, chirp_z):
    t_plot_gauss, env = plot_e_gauss(T_o, T_z)
    cos_chirp = plot_chirp(T_o, T_z, chirp_z)
    pulso_gaussiano = cos_chirp * env
    To_plot = int(T_o.to(ureg('picosecond')).magnitude)
    Tz_plot = int(T_z.to(ureg('picosecond')).magnitude)
    plt.figure(1, dpi=80)
    plt.plot(t_plot_gauss, pulso_gaussiano)
    plt.grid(True)
    plt.figure(1).suptitle('Pulso gaussiano', fontsize=20)
    plt.xlabel('Tiempo en picosegundos', fontsize=18)
    plt.ylabel('Potencia normalizada', fontsize=16)
    plt.xlim([-3*To_plot, 3*To_plot])
    plt.ylim([-2.5, 2.5])

In [10]:
def plot(wbitrate, wchirp_o, wlambda_o, wdispersion_cromatica, wlongitud):
    bitrate = wbitrate * ureg('gigabit/second')
    chirp_o = wchirp_o * ureg('dimensionless')
    lambda_o = wlambda_o * ureg('nanometer')
    dispersion_cromatica = wdispersion_cromatica * ureg('picosecond / (nanometer * kilometer)')
    longitud = wlongitud * ureg('kilometer')
    ###################################################
    T_o, T_z, Tz_To, beta_2, chirp_z, L_Dispersion = calculo_parametros(bitrate, lambda_o , dispersion_cromatica, chirp_o, longitud)    
    ####################################################
    #plot_pulso_gaussiano(T_o, T_z, chirp_o, chirp_z)
    t_plot_gauss, env = plot_e_gauss(T_o, T_z)
    cos_chirp = plot_chirp(T_o, T_z, chirp_z)
    pulso_gaussiano = cos_chirp * env
    To_plot = int(T_o.to(ureg('picosecond')).magnitude)
    Tz_plot = int(T_z.to(ureg('picosecond')).magnitude)
    plt.figure(1, dpi=80)
    plt.plot(t_plot_gauss, pulso_gaussiano)
    plt.grid(True)
    plt.figure(1).suptitle('Pulso gaussiano', fontsize=20)
    plt.xlabel('Tiempo en picosegundos', fontsize=18)
    plt.ylabel('Potencia normalizada', fontsize=16)
    plt.xlim([-3*To_plot, 3*To_plot])
    plt.ylim([-2.5, 2.5])
    plt.show()
    print('Beta2 = ', beta_2.to(ureg('picosecond ** 2 / kilometer')))
    print('To = ', T_o.to(ureg('picosecond')))
    print('Long Dispersion = ', L_Dispersion)
    print('Tz =', T_z.to(ureg('picosecond')))
    print('Tz/To =', Tz_To)
    print('Chirp =', chirp_z)

wbitrate = widgets.FloatSlider(min=1,max=10,step=1,value=10, description='Bitrate Gbps')
wchirp_o = widgets.IntSlider(min=-5,max=5,step=1,value=0, description='Chirp inicial')
wlambda_o = widgets.IntSlider(min=850,max=1550,step=10,value=15500, description='λo nm')
wdispersion_cromatica =  widgets.IntSlider(min=0,max=20,step=1,value=18, description='D ps/(nm·km)')
wlongitud = widgets.IntSlider(min=0,max=200,step=1,value=50, description='Longitud km')

interactive_plot = interactive(plot, wbitrate = wbitrate, wchirp_o = wchirp_o, wlambda_o = wlambda_o, wdispersion_cromatica = wdispersion_cromatica, wlongitud = wlongitud)
output = interactive_plot.children[-1]
output.layout.height = '1000px'
interactive_plot

interactive(children=(FloatSlider(value=10.0, description='Bitrate Gbps', max=10.0, min=1.0, step=1.0), IntSli…